In [14]:
import pandas as pd

def read_sequences_from_file(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith('>'):
                sequences.append(line.strip())
    return sequences

def find_first_sequence_with_gataa(sequences):
    for sequence in sequences:
        if 'ACCGGAAG' in sequence:
            return sequence
    return None

def generate_substrings(base_str):
    substrings = []
    length = len(base_str)
    for i in range(length):
        for j in range(i+1, length+1):
            if base_str[i:j] and len(base_str[i:j]) != length:
                substrings.append(base_str[i:j])
    return sorted(set(substrings), key=len, reverse=True)

def align_sequences(sequences, reference_sequence):
    ref_index = reference_sequence.find('ACCGGAAG')
    substrings = generate_substrings('ACCGGAAG')

    columns = [i for i in range(-7, 15)]  
    df = pd.DataFrame(columns=columns)

    for sequence in sequences:
        aligned = False
        sequence = sequence.replace('.', '')
        if 'ACCGGAAG' in sequence:
            idx = sequence.find('ACCGGAAG')
            aligned = True
            entire = True
        else:
            for substring in substrings:
                if substring in sequence:
                    idx = sequence.find(substring)
                    ref_index2 = reference_sequence.find(substring)
                    entire = False
                    aligned = True
                    break

        if aligned and entire:
            offset = ref_index - idx
            indices = range(offset, offset + len(sequence))
            df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
            
        if aligned and not entire:
            offset = ref_index2 - idx
            indices = range(offset, offset + len(sequence))
            df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
        

    return df

input_file_path = '/Users/bristi/Desktop/Design Project/Working-with-TF/ETV-5/bin_1/bin_1.fasta'
output_file_path = 'aligned_sequences2.csv'

sequences = read_sequences_from_file(input_file_path)

reference_sequence = find_first_sequence_with_gataa(sequences)
if not reference_sequence:
    raise ValueError("No sequence containing 'ACCGGAAG' was found in the file.")

df_aligned = align_sequences(sequences, reference_sequence)

/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_4895/1509766517.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_4895/1509766517.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_4895/1509766517.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(list(sequence), index=indices), ignore_index=True)
/var/folders/z1/0b4fr5_n0fg0z6szcfq9km100000gn/T/ipykernel_4895/1509766517.py:57: FutureWarning: The frame.append method 

In [15]:
df_aligned

,-7,-6,-5,-4,-3,-2,-1,0,1,2,...,5,6,7,8,9,10,11,12,13,14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,C,C,...,A,A,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,A,T,T,C,C,...,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,A,C,T,C,C,...,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,A,C,A,T,C,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,C,...,A,A,G,T,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747,NaN,NaN,NaN,C,A,G,A,T,C,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5748,NaN,NaN,NaN,NaN,NaN,G,A,T,C,C,...,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5749,NaN,NaN,NaN,NaN,NaN,T,C,C,C,C,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5750,NaN,NaN,NaN,NaN,NaN,G,C,G,A,C,...,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Generating txt file for weblogo

In [16]:
columns_to_extract = [col for col in range(0, 8) if col in df_aligned.columns]
extracted_data = df_aligned[columns_to_extract]
extracted_data = extracted_data.fillna('-')

formatted_data = extracted_data.apply(lambda row: ''.join(row.astype(str)), axis=1)

output_file_path = 'ETV_5_BIN1_aligned.txt'
with open(output_file_path, 'w') as file:
    for line in formatted_data:
        file.write(line + '\n')